# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: XXXX XXXX
#### Student ID: 000000

Date: XXXX

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries 

In [42]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

## Task 2. Generating Feature Representations for Job Advertisement Descriptions

...... Sections and code blocks on buidling different document feature represetations


<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [43]:
# Load the vocabulary
with open("vocab.txt", "r") as f:
    vocab = f.read().splitlines()

# Create a dictionary mapping each word in the vocabulary to a unique integer
word_to_index = {word: idx for idx, word in enumerate(vocab)}

# Load the job advertisements dataset
job_ads = pd.read_csv("processed_job_ads.csv")

# Convert the 'Tokenized Description' column from string to list
job_ads["Tokenized Description"] = job_ads["Tokenized Description"].apply(lambda x: eval(x))

# Convert each job advertisement description into a sparse count vector representation
count_vectors = []

for _, row in job_ads.iterrows():
    vector = {}
    for word in row["Tokenized Description"]:
        if word in word_to_index:
            vector[word_to_index[word]] = vector.get(word_to_index[word], 0) + 1
    sorted_vector = sorted(vector.items(), key=lambda x: x[0])
    count_vectors.append((row["Webindex"], sorted_vector))

# Display the first few count vectors for review
count_vectors[:5]

[(68802053, []),
 (70757636, []),
 (71356489, []),
 (69073629, []),
 (70656648, [])]

In [44]:
# Displaying the first few words from the vocabulary and a few tokenized descriptions for review
vocab[:10], job_ads["Tokenized Description"].head().tolist()

(['aap:0',
  'aaron:1',
  'aat:2',
  'abb:3',
  'abenefit:4',
  'aberdeen:5',
  'abi:6',
  'abilities:7',
  'abreast:8',
  'abroad:9'],
 [['market',
   'retail',
   'rapid',
   'growth',
   'due',
   'expansion',
   'add',
   'financial',
   'planning',
   'analyst',
   'central',
   'central',
   'london',
   'fantastic',
   'newly',
   'created',
   'driving',
   'forward',
   'financial',
   'planning',
   'analysis',
   'group',
   'reporting',
   'directly',
   'head',
   'fp',
   'assist',
   'revenue',
   'analysis',
   'product',
   'channel',
   'region',
   'commercial',
   'input',
   'review',
   'cases',
   'presenting',
   'proposals',
   'approval',
   'develop',
   'endtoend',
   'planning',
   'cycle',
   'processes',
   'lead',
   'regional',
   'planning',
   'processes',
   'ensure',
   'completeness',
   'key',
   'channels',
   'products',
   'finance',
   'year',
   'strategic',
   'plan',
   'addition',
   'globally',
   'regions',
   'develop',
   'capital',
  

In [45]:
# Correcting the vocabulary by stripping the indices
vocab_cleaned = [word.split(":")[0] for word in vocab]

# Update the dictionary mapping each word in the cleaned vocabulary to a unique integer
word_to_index_cleaned = {word: idx for idx, word in enumerate(vocab_cleaned)}

# Convert each job advertisement description into a sparse count vector representation based on the cleaned vocabulary
count_vectors_cleaned = []

for _, row in job_ads.iterrows():
    vector = {}
    for word in row["Tokenized Description"]:
        if word in word_to_index_cleaned:
            vector[word_to_index_cleaned[word]] = vector.get(word_to_index_cleaned[word], 0) + 1
    sorted_vector = sorted(vector.items(), key=lambda x: x[0])
    count_vectors_cleaned.append((row["Webindex"], sorted_vector))

# Display the first few count vectors for review after correction
count_vectors_cleaned[:5]

[(68802053,
  [(13, 1),
   (17, 1),
   (64, 1),
   (73, 1),
   (77, 1),
   (154, 1),
   (226, 2),
   (227, 2),
   (263, 1),
   (278, 1),
   (343, 1),
   (378, 1),
   (495, 1),
   (644, 1),
   (659, 1),
   (669, 1),
   (705, 1),
   (732, 2),
   (759, 1),
   (760, 1),
   (803, 1),
   (891, 1),
   (942, 1),
   (985, 1),
   (1136, 1),
   (1178, 1),
   (1316, 2),
   (1354, 1),
   (1367, 1),
   (1449, 1),
   (1457, 1),
   (1565, 2),
   (1585, 1),
   (1616, 1),
   (1737, 1),
   (1801, 1),
   (1823, 1),
   (1840, 1),
   (1842, 2),
   (1857, 1),
   (1922, 2),
   (1928, 1),
   (1932, 1),
   (2042, 1),
   (2078, 1),
   (2084, 1),
   (2137, 2),
   (2145, 1),
   (2401, 1),
   (2458, 1),
   (2459, 1),
   (2474, 1),
   (2501, 1),
   (2589, 1),
   (2641, 1),
   (2699, 1),
   (2751, 1),
   (2835, 1),
   (3102, 1),
   (3431, 1),
   (3487, 2),
   (3491, 4),
   (3522, 1),
   (3609, 1),
   (3646, 1),
   (3665, 2),
   (3673, 1),
   (3676, 1),
   (3722, 1),
   (3788, 1),
   (3824, 1),
   (3912, 1),
   (3913,

### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [46]:
# Saving the count vectors to count_vectors.txt
with open("count_vectors.txt", "w") as f:
    for webindex, vector in count_vectors_cleaned:
        vector_str = ",".join([f"{idx}:{freq}" for idx, freq in vector])
        f.write(f"#{webindex},{vector_str}\n")

## Task 3. Job Advertisement Classification

...... Sections and code blocks on buidling classification models based on different document feature represetations. 
Detailed comparsions and evaluations on different models to answer each question as per specification. 

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [47]:
# Load the dataset
df_job_ads = pd.read_csv('processed_job_ads.csv')
df_job_ads.head()


,Title,Webindex,Company,Description,Category,Tokenized Description
0,FP&A Blue Chip,68802053,Hays Senior Finance,A market leading retail business is going thro...,Accounting_Finance,"['market', 'retail', 'rapid', 'growth', 'due',..."
1,Part time Management Accountant,70757636,FS2 UK Ltd,You will be responsible for the efficient runn...,Accounting_Finance,"['responsible', 'efficient', 'running', 'accou..."
2,IFA EMPLOYED,71356489,Clark James Ltd,Role The purpose of the role is to provide adv...,Accounting_Finance,"['purpose', 'advice', 'telephone', 'leads', 's..."
3,Finance Manager,69073629,Accountancy Action Ltd,"Excellent opportunity to join our client, an e...",Accounting_Finance,"['expanding', 'recruit', 'aca', 'qualified', '..."
4,Management Accountant,70656648,Alexander Lloyd,Our client offers a interesting opportunity fo...,Accounting_Finance,"['offers', 'interesting', 'part', 'qualified',..."


In [48]:
# Load vocabulary
with open('vocab.txt', 'r') as f:
    vocab = f.read().splitlines()

# Load a few lines from the count vectors file to inspect its structure
sample_count_vectors = pd.read_csv('count_vectors.txt', sep=" ", header=None, nrows=5)
sample_count_vectors

,0
0,"#68802053,13:1,17:1,64:1,73:1,77:1,154:1,226:2..."
1,"#70757636,16:1,34:1,35:1,36:2,40:1,135:1,225:1..."
2,"#71356489,77:1,89:1,130:3,135:1,241:2,274:1,42..."
3,"#69073629,13:2,33:1,35:1,36:1,240:1,353:1,600:..."
4,"#70656648,13:1,16:2,32:1,33:2,34:2,36:1,39:1,4..."


Description as Data for Learning

In [49]:
# Extract Webindex and count vectors from the sparse representation
web_indices = []
dense_count_vectors = np.zeros((sample_count_vectors.shape[0], len(vocab)))

for idx, row in enumerate(sample_count_vectors[0]):
    items = row.split(',')
    web_indices.append(items[0][1:])  # Extract Webindex (excluding the '#')
    
    for item in items[1:]:
        word_idx, count = item.split(':')
        dense_count_vectors[idx, int(word_idx)] = int(count)

# Convert to DataFrame
df_dense_count_vectors = pd.DataFrame(dense_count_vectors, columns=vocab)
df_dense_count_vectors['Webindex'] = web_indices

df_dense_count_vectors.head()

,aap:0,aaron:1,aat:2,abb:3,abenefit:4,aberdeen:5,abi:6,abilities:7,abreast:8,abroad:9,...,yeovil:5159,yn:5160,york:5161,yorkshire:5162,youmust:5163,young:5164,younger:5165,yrs:5166,zest:5167,Webindex
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68802053
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70757636
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71356489
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69073629
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70656648


In [50]:
# Load the entire count vectors dataset
full_count_vectors = pd.read_csv('count_vectors.txt', sep=" ", header=None)

# Extract Webindex and count vectors from the sparse representation
web_indices_full = []
dense_count_vectors_full = np.zeros((full_count_vectors.shape[0], len(vocab)))

for idx, row in enumerate(full_count_vectors[0]):
    items = row.split(',')
    web_indices_full.append(items[0][1:])  # Extract Webindex (excluding the '#')
    
    for item in items[1:]:
        word_idx, count = item.split(':')
        dense_count_vectors_full[idx, int(word_idx)] = int(count)

# Convert to DataFrame
df_dense_count_vectors_full = pd.DataFrame(dense_count_vectors_full, columns=vocab)
df_dense_count_vectors_full['Webindex'] = web_indices_full

df_dense_count_vectors_full.head()

,aap:0,aaron:1,aat:2,abb:3,abenefit:4,aberdeen:5,abi:6,abilities:7,abreast:8,abroad:9,...,yeovil:5159,yn:5160,york:5161,yorkshire:5162,youmust:5163,young:5164,younger:5165,yrs:5166,zest:5167,Webindex
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68802053
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70757636
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71356489
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69073629
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70656648


In [51]:
# Convert Webindex to int for both dataframes
df_dense_count_vectors_full['Webindex'] = df_dense_count_vectors_full['Webindex'].astype(int)
df_job_ads['Webindex'] = df_job_ads['Webindex'].astype(int)

# Merge the datasets based on Webindex
merged_data = pd.merge(df_dense_count_vectors_full, df_job_ads[['Webindex', 'Category']], on='Webindex', how='inner')

# Splitting the dataset into features and target
X = merged_data.drop(columns=['Webindex', 'Category'])
y = merged_data['Category']

X.shape, y.shape

((776, 5168), (776,))

In [52]:
# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(lr_model, X, y, cv=5, scoring='accuracy')

# Calculate the average accuracy from the 5-fold cross-validation
avg_accuracy = np.mean(cv_scores)
avg_accuracy

0.8852936311000826

Title as Data for Learning

In [53]:
# Initialize CountVectorizer using the provided vocabulary
vectorizer = CountVectorizer(vocabulary=vocab, lowercase=True, token_pattern=r'\b\w+\b')

# Generate count vectors for job titles
title_vectors = vectorizer.transform(df_job_ads['Title']).toarray()

# Convert to DataFrame
df_title_vectors = pd.DataFrame(title_vectors, columns=vocab)

df_title_vectors.head()

,aap:0,aaron:1,aat:2,abb:3,abenefit:4,aberdeen:5,abi:6,abilities:7,abreast:8,abroad:9,...,years:5158,yeovil:5159,yn:5160,york:5161,yorkshire:5162,youmust:5163,young:5164,younger:5165,yrs:5166,zest:5167
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
# Perform 5-fold cross-validation using title vectors as features
title_cv_scores = cross_val_score(lr_model, df_title_vectors, y, cv=5, scoring='accuracy')

# Calculate the average accuracy from the 5-fold cross-validation for titles
avg_title_accuracy = np.mean(title_cv_scores)
avg_title_accuracy

0.2976840363937138

Combined Representations

In [55]:
# Concatenate the count vectors of titles and descriptions
combined_vectors = pd.concat([df_title_vectors, X.reset_index(drop=True)], axis=1)

combined_vectors.head()

,aap:0,aaron:1,aat:2,abb:3,abenefit:4,aberdeen:5,abi:6,abilities:7,abreast:8,abroad:9,...,years:5158,yeovil:5159,yn:5160,york:5161,yorkshire:5162,youmust:5163,young:5164,younger:5165,yrs:5166,zest:5167
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# Perform 5-fold cross-validation using combined vectors as features
combined_cv_scores = cross_val_score(lr_model, combined_vectors, y, cv=5, scoring='accuracy')

# Calculate the average accuracy from the 5-fold cross-validation for combined vectors
avg_combined_accuracy = np.mean(combined_cv_scores)
avg_combined_accuracy

0.8852936311000826

## Summary
Give a short summary and anything you would like to talk about the assessment tasks here.

Q1: Language model comparisons
- I utilized count vectors generated from job advertisement descriptions as the feature representations.
- Using a logistic regression model and a 5-fold cross-validation approach, the model achieved an average accuracy of approximately 88.53% when classifying job categories based on descriptions.

Q2: Does more information provide higher accuracy?

Title only:         29.77%

Description only:   88.53%

Combined:           88.53%


## Couple of notes for all code blocks in this notebook
- please provide proper comment on your code
- Please re-start and run all cells to make sure codes are runable and include your output in the submission.   
<span style="color: red"> This markdown block can be removed once the task is completed. </span>